# Allrecipes
Set of some tasks related to scraping allrecipes.com  
It does not aim to be pretty, it aims to be usefull

In [1]:
import urllib.request

import sys
import time
import random
import json

from bs4 import BeautifulSoup

import multiprocessing as mp

In [2]:
links = []
with open('links.txt') as f:
    links = list(map(lambda x: x.strip(), f.readlines()))

In [3]:
# allrecipes get directions
def directions_allrecipes(soup):
    temp = list(map(lambda x: x.get_text().strip(), soup.find_all('span', 'recipe-directions__list--item')))
    temp = list(filter(lambda x: x != '', temp))
    return temp

In [4]:
# allrecipes get ingredients
def ingredients_allrecipes(soup):
    temp = list(map(lambda x: x.get_text().strip(), soup.find_all('span', 'recipe-ingred_txt')))
    temp = list(filter(lambda x: x != '' and x != 'Add all ingredients to list', temp))
    return temp

In [5]:
# allrecipes get title
def title_allrecipes(soup):
    return list(map(lambda x: x.get_text().strip(), soup.find_all('h1','recipe-summary__h1')))[0]

In [6]:
## the proper function to be used in future and in framework
def save_recipe(step, url, retrieve_ingredients, retrieve_directions, retrieve_title, path="./", filename_prefix="", filename_suffix=""):
    try:
        start = time.time()
        
        # get page
        html_doc = urllib.request.urlopen(url).read().decode('utf-8')
        # parse
        soup = BeautifulSoup(html_doc, 'html.parser')
        # retrive information and save to dictionary
        title = retrieve_title(soup)
        recipe = dict()
        recipe['title'] = title
        recipe['ingredients'] = retrieve_ingredients(soup)
        recipe['directions'] = retrieve_directions(soup)
        # save to file
        with open(path+'/'+filename_prefix+title.lower().replace(' ','_')+filename_suffix+'.json', 'w+') as f:
            f.write(json.dumps(recipe))
            
        end = time.time()
        elapsed = end-start
        
        print("Step:\t", step, "Time:\t", elapsed)
    
    except:
        print("Unable to get recipe from:\t", url)
        #for s in sys.exc_info():
        #    print(s)

## Missing files
Find files which were not downloaded

In [7]:
with open('crawled.txt') as f:
    links = list(map(lambda x: x.strip(), f.readlines()))
    
ids = list(map(lambda x: int(x.split('-')[0]), links))

with open('links.txt') as f:
    alllinks = list(map(lambda x: x.strip(), f.readlines()))
    
missing = []
for i in range(len(alllinks)):
    if i not in ids:
        missing.append(alllinks[i])

with open('missing.txt', 'w+') as f:
    for m in missing:
        f.write(m + '\n')

# Adding links to recipes
Since allrecipes.com was scraped in very early phase, the original files had not contained source urls, so to avoid processing this wevsite again, it was decided to add links from links list to recipes, matching by recipe title.

In [8]:
from os import listdir

In [9]:
source_dir = 'recipes_old'
target_dir = 'recipes'

In [11]:
links = []
with open('links.txt') as f:
    links = list(map(lambda x: x.strip(), f.readlines()))

In [12]:
filenames = listdir(source_dir)
filenames.remove('.ipynb_checkpoints')
filenames.sort()
linknames = list(map(lambda x: (x.split('/')[-2], x), links))
linknames_dict = dict(linknames)

In [13]:
matching = []
notmatching = 0
for f in filenames:
    with open(source_dir + '/' + f) as jf:
        rec = json.load(jf)
    name = rec['title'].lower().replace(' ', '-').replace('\'', '').replace(',', '').replace('"', '').replace('®', '').replace('™', '')
    name = name.replace('!', '').replace('(','').replace(')','').replace(':', '').replace('.','').replace('&-', '').replace('---', '-').replace('--', '-')
    if name in linknames_dict:
        rec['link'] = linknames_dict[name]
        matching.append(rec)
    else:
        #print(name)
        notmatching += 1
        #pass
    
notmatching

281

In [14]:
len(matching)

62022

In [15]:
for i, m in enumerate(matching):
    str_id = '0' * (8 - len(str(i))) + str(i)
    with open(target_dir + '/' + str_id + '-' + m['title'].lower().replace(' ','_') + '.json', 'w+') as wf:
        wf.write(json.dumps(m))

In [16]:
%%bash
ls recipes | wc -l

62022
